In [14]:
import pandas as pd
from tabula import read_pdf
from PyPDF2 import PdfFileReader

#### **VietinBank Statement**

In [2]:
# Read Vietinbank statement file
vtb_path = 'data/VietinBank.pdf'
vtb = read_pdf(vtb_path, pages='all') # As there are only over 2000 pages so we can read them all at once

In [5]:
vtb_full = vtb[0].copy()
vtb_full.columns = vtb_full.iloc[0,:]
vtb_full.drop(index=[0,2], axis=0, inplace=True)
vtb_full = vtb_full.loc[:, vtb_full.columns.notna()]
vtb_full.dropna(axis=0, how='all', inplace=True)

for i in range(1,len(vtb)):
    vtb[i].loc[len(vtb[i])] = vtb[i].columns.values
    vtb[i].columns = vtb_full.columns
    vtb_full = pd.concat([vtb_full, vtb[i]], ignore_index=True)
    print(f'Concatnated {i+1} files.')

Concatnated 2 files.
Concatnated 3 files.
Concatnated 4 files.
Concatnated 5 files.
Concatnated 6 files.
Concatnated 7 files.
Concatnated 8 files.
Concatnated 9 files.
Concatnated 10 files.
Concatnated 11 files.
Concatnated 12 files.
Concatnated 13 files.
Concatnated 14 files.
Concatnated 15 files.
Concatnated 16 files.
Concatnated 17 files.
Concatnated 18 files.
Concatnated 19 files.
Concatnated 20 files.
Concatnated 21 files.
Concatnated 22 files.
Concatnated 23 files.
Concatnated 24 files.
Concatnated 25 files.
Concatnated 26 files.
Concatnated 27 files.
Concatnated 28 files.
Concatnated 29 files.
Concatnated 30 files.
Concatnated 31 files.
Concatnated 32 files.
Concatnated 33 files.
Concatnated 34 files.
Concatnated 35 files.
Concatnated 36 files.
Concatnated 37 files.
Concatnated 38 files.
Concatnated 39 files.
Concatnated 40 files.
Concatnated 41 files.
Concatnated 42 files.
Concatnated 43 files.
Concatnated 44 files.
Concatnated 45 files.
Concatnated 46 files.
Concatnated 47 fil

In [6]:
vtb_full.head()

,STT,Ngày GD,Mô tả giao dịch,Số tiền,Tên đối ứng
0,1,10/09/2024\r12:01:29,CT nhanh 247 den: TRAN TIEN ANH chuyen\rtien u...,300.000,TRAN TIEN ANH – A/C
1,2,10/09/2024\r12:01:46,CT nhanh 247 den: Phuong Quynh Le chuyen\rtien,100.000,LE PHUONG QUYNH – A/C
2,3,10/09/2024\r13:33:47,TRAN TRONG SON Chuyen tien ung ho\r(CT1111),100.000,TRAN TRONG SON – A/C
3,4,10/09/2024\r13:34:31,NGUYEN THU BINH chuyen tien ung ho\r(CT1111),100.000,NGUYEN THU BINH – A/C
4,5,10/09/2024\r13:37:09,Chuyen tien den tu NAPAS Noi dung: NGUYEN\rTHU...,100.000,VND-TGTT-NGUYEN THU BINH – A/C


In [10]:
vtb_full.set_index(keys='STT', inplace=True)

# define lambda function to remove '\r' in string
string_clean = lambda x: str(x).replace('\r', ' ')

# clean and transform the columns
vtb_full.loc[:, 'Ngày GD'] = pd.to_datetime(vtb_full['Ngày GD'].apply(string_clean), format='%d/%m/%Y %H:%M:%S')
vtb_full.loc[:, 'Mô tả giao dịch'] = vtb_full['Mô tả giao dịch'].apply(string_clean)
vtb_full.loc[:, 'Tên đối ứng'] = vtb_full['Mô tả giao dịch'].apply(string_clean)
vtb_full.loc[:, 'Số tiền'] = vtb_full['Số tiền'].apply(lambda x: int(str(x).replace('.','')))

In [12]:
# Save the output to a csv file
vtb_full.to_csv('data/VietinBank_statement.csv')

#### **VietcomBank Account Bank Statement**

In [18]:
vcb_path = 'data/VietcomBank.pdf'

# Test reading VietcomBank statements file
vcb_test = read_pdf(vcb_path, pages=1, stream=True) # As the tables in the file are borderless, setting stream=True helps focus on text alignment
vcb_test[0].head(10)

,Ngày GD/,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Số tiền ghi nợ/,Số tiền ghi có/,Số dư/,Nội dung chi tiết/
1,TNX Date,NaN,NaN,NaN,NaN
2,NaN,Debit,Credit,Balance,Transactions in detail
3,Số CT/ Doc No,NaN,NaN,NaN,NaN
4,01/09/2024,NaN,NaN,NaN,NaN
5,NaN,NaN,50.000,NaN,292976.010924.013647.xin cam on
6,5213.45946,NaN,NaN,NaN,NaN
7,01/09/2024,NaN,NaN,NaN,NaN
8,NaN,NaN,20.000,NaN,VCB.CTDK.31/03/2024.ADIDA PHAT. CT tu
9,5090.85797,NaN,NaN,NaN,NaN


In [ ]:
# Clean the data in batches as the pdf file has over 12 thousand pages

def count_pdf_pages(file_path):
    """Function returns the number of pages in a pdf file"""
    with open(file_path, 'rb') as file:
        reader = PdfFileReader(file)
        return reader.numPages
    
def cleaning_vcb(df: pd.DataFrame):
    # skip the first 4 rows of headers
    df = df[4:]

    # rename and drop unnecessary columns
    df.columns = ['transaction_date', 'debit', 'credit', 'balance', 'detail']
    df = df.drop(['debit', 'balance'], axis=1)

    # add column 'transaction_no'
    cleaned_df = pd.DataFrame(columns= df.columns.insert(0, 'transaction_no'))

    # set variables for each column value
    _date = _transactionNo = _credit = None
    _detail = [] # set list as a transaction detail may have multiple rows
    row_count = 0

    for _, row in df.iterrows():
        transaction_date, credit, detail = row
        if pd.notna(transaction_date):
            if '/' in transaction_date:
                # Append row to df and reset the variables when reaching a date value in column 'Ngày GD'
                if _transactionNo and _date and _credit and _transactionNo:
                    cleaned_df.loc[row_count] = [_transactionNo, _date, _credit, ' '.join(_detail)]
                    _date = _transactionNo = _credit = None
                    _detail = []
                    row_count += 1

                _date = transaction_date
            elif '.' in transaction_date:
                _transactionNo = transaction_date
        
        if pd.notna(credit):
            _credit = credit

        if pd.notna(detail):
            _detail.append(detail)

    # Handling the last transaction after the loop
    if _transactionNo and _date and _credit:
        cleaned_df.loc[row_count] = [_transactionNo, _date, _credit, ' '.join(_detail)]

    return cleaned_df

# Loop through each 500 pages to clean and concatnate the full dataframe
vcb_full = pd.DataFrame()

num_pages = count_pdf_pages(vcb_path)
per_pages = 500
loops = num_pages // per_pages

for i in range(loops + 1):

    from_page = per_pages * i + 1
    to_page = (per_pages * (i + 1)) if i <= loops else (from_page + (num_pages % per_pages) - 1) # Handling the remaining pages in the last loop
    print(f'\nStart reading from page {from_page} to page {to_page}...')
    vcb = read_pdf(vcb_path, pages=list(range(from_page, to_page + 1)), stream=True)

    for page in range(len(vcb)):
        cleanned_df = cleaning_vcb(vcb[page])
        vcb_full = pd.concat([vcb_full, cleanned_df], ignore_index=True)
        print(f'Concatnated {page + from_page} pages.')

# Save the output to a csv file
vcb_full.to_csv('data/Vietcombank_statement.csv', index=False)